In [ ]:
# !which python
# import sys
# print(sys.executable)

!python -m pip install --upgrade pip
# !python -m pip uninstall pycrypto --yes
!python -m pip install pycryptodome
# !python -m easy_install pycryptodome  # uncomment this if the above pip install did not work! 
!python -m pip install Pillow
!python -m jupyter nbextension enable --py widgetsnbextension --sys-prefix
print('Ready to Go!')

# Symmetric Key Encryption: AES
AES, standing for *__A__dvanced __E__ncryption __S__ystem*, is today's (2010's!) standard symmetric-key cipher. 

<div class="alert alert-block alert-info">*Recall:* cipher == cryptosystem == a suite of cryptographic algorithms e.g. encryption/decryption achieving a security service, here: confidentiality.</div>

This set of exercises is to get you familiar with AES, some of its properties, including modes of operation and **see (literally)** how dangerous it is if used wrongly! 

### AES Initial Questions

 - __Q0:__ As you may recall (from the book, of course!), AES won a public competition by NIST to become the standard symmetric cipher in 2000. Visit the __[competition website](https://competitions.cr.yp.to/aes.html)__ and find out what were the merits of selection and the competitors.  
 
 - __Q1:__ what is the block size of AES (in terms of bits and bytes) 
 
 - __Q2:__ what is the possible key-lengths of AES? (in terms of bits and bytes)
 
 - __Q3:__ compared to DES, how many times longer does a brute-force attack take on AES even when the shortest key-length of AES is used?
 
 - __Q4:__ name the 5 common modes of operation of AES.
 
 - __Q5:__ To give you the confidence that we are indeed in the realm of current standards of the day, find out the key-length and mode of operation that the following websites use:
  - https://www.google.co.uk
  - https://www.nwolb.com
  - https://www.security.hsbc.co.uk
  - https://bank.barclays.co.uk
  - a website of your choice!
  
 *Hint: you can simply use your browser for this exercise. For example, in Firefox, click on the green lock appearing next to the link, and then click on "more information". In chrome, you can find such information from the developer tools (DevTool), e.g. by using the shortkey `Ctrl+Shift+I`. Screenshots are provided to you through QM+:*
 ![firefox snapshot](firefox_example.png)
 ![chrome snapshot](chrome_example.png)

## Dangers of ECB
You should notice that one of the operation modes of AES is nowhere to be found: ECB (if you can find it used for confidentiality service of a website, bring it to my attention for a bonus point!). In what follows, hopefully, we will **see** why! (can you guess?)

We will encrypt a plaintext, which is an image. Note: (of course) plaintext does not mean text: all data in our information systems are just zeros and ones, audio, video, documents, code, anything. Again, of course! So unlike the "historical" ciphers, any modern cipher should be able to take any bytes as input, irrespective of what it represents).  

In [ ]:
%reset -f
from IPython.display import display
import PIL.Image
from Crypto.Cipher import AES
import ipywidgets as widgets

im = PIL.Image.open('plaintext.ppm')
display(im)

with open('plaintext.ppm','rb') as inputfilehandle:
    header = inputfilehandle.read(54)
    body = inputfilehandle.read()

# key
input_key = widgets.Text(
    value = '0', # initial inside text
    placeholder = 'Enter AES key (in hex)', # inside text when cleared 
    description = 'AES Key (in hex):', # text before the text-input widget
    style = {'description_width': 'initial'})

def aes_encrypt(key):
    
    try:
        key = bytes.fromhex(key)
        MY_AES_ECB_CIPHER = AES.new(key, AES.MODE_ECB)
        body_enc = MY_AES_ECB_CIPHER.encrypt(body)
        ciphertext = header+body_enc
        with open('ciphertext.ppm','wb') as outputfilehandle:
            outputfilehandle.write(ciphertext)
        im_enc = PIL.Image.open('ciphertext.ppm')
        display(im_enc)
    except ValueError as e:
        print(e) # print the specific error raised
    except Exception as err:
        print(err)
        raise
        
out = widgets.interactive_output(aes_encrypt, {'key': input_key})
display(input_key, out)

 - __Q0:__ Pass a valid (HEX) key for the 128-bit key length. 
 
 - __Q1:__ Do you see the ciphertext? Explain why confidentiality (that is the service that encryption should have provided) visibly fails!
 
 - __Q2:__ Try the other two longer key modes. Does it remedy the problem?
 
 - __Q3:__ Does this mean AES is broken!?

  - __Q4:__ Let us try a different mode of operation, say CBC. Complete the following code and see the ciphertext. Note: there is a missing argument for the CBC mode, what is it?

In [ ]:
%reset -f
from IPython.display import display
import PIL.Image
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes


import ipywidgets as widgets

im = PIL.Image.open('plaintext.ppm')
display(im)

with open('plaintext.ppm','rb') as inputfilehandle:
    header = inputfilehandle.read(54)
    body = inputfilehandle.read()

# key
input_key = widgets.Text(
    value = '0', # initial inside text
    placeholder = 'Enter AES key (in hex)', # inside text when cleared 
    description = 'AES Key (in hex):', # text before the text-input widget
    style = {'description_width': 'initial'})

def aes_encrypt(key):
    
    try:
        key = bytes.fromhex(key)
        MY_AES_CBC_CIPHER = AES.new(key, AES.MODE_CBC) # oops! something seems missing here!
        # IV = get_random_bytes(AES.block_size)
        # MY_AES_CBC_CIPHER = AES.new(key, AES.MODE_CBC, IV=IV)
        body_enc = MY_AES_CBC_CIPHER.encrypt(body)
        ciphertext = header+body_enc
        with open('ciphertext.ppm','wb') as outputfilehandle:
            outputfilehandle.write(ciphertext)
        im_enc = PIL.Image.open('ciphertext.ppm')
        display(im_enc)
    except ValueError as e:
        print(e) # print the specific error raised
    except Exception as err:
        print(err)
        raise
        
out = widgets.interactive_output(aes_encrypt, {'key': input_key})
display(input_key, out)

 - __Q5:__ what is the effect of AES CBC mode on the ciphertext?
 
 - __Q6:__ Try the same exercise on another "plaintext" in the following. You will encounter a different error. Fix it!

In [ ]:
%reset -f
from IPython.display import display
import PIL.Image
from Crypto.Cipher import AES

import ipywidgets as widgets
import io

# key
input_key = widgets.Text(
    value = '00000000000000000000000000000000', # initial inside text
    placeholder = 'Enter your AES key', # inside text when cleared 
    description = 'AES Key:', # text before the text-input widget
    style = {'description_width': 'initial'})

def aes_encrypt(key, plaintext):

    MY_AES_ECB_CIPHER = AES.new(key, AES.MODE_ECB)
    ciphertext = MY_AES_ECB_CIPHER.encrypt(plaintext)
    return ciphertext
    

        
def aes_encrypt_reddit(key):

    key = bytes.fromhex(key)
    im = PIL.Image.open('reddit.ppm')
    display(im)

    with open('reddit.ppm','rb') as inputfilehandle:
        header = inputfilehandle.read(55)
        body = inputfilehandle.read()

    body_enc = aes_encrypt(key, body)
    with open('reddit_enc_aes.ppm','wb') as outputfilehandle:
        outputfilehandle.write(header+body_enc)

    im_enc = PIL.Image.open('reddit_enc_aes.ppm')
    display(im_enc)

out = widgets.interactive_output(aes_encrypt_reddit, {'key': input_key})
display(input_key, out)

In [ ]:
# ask for a hint if stuck!